In [6]:
import os
import sys
import glob

sys.path.append(os.path.join(".."))
from tta_uia_segmentation.src.preprocessing.utils import get_filepaths_raw, get_filepaths_preprocessed

In [7]:
a = get_filepaths_preprocessed(path_to_dir="/scratch_net/biwidl319/jbermeo/data/preprocessed/1_resampled/USZ")

In [8]:
len(a)

62

In [5]:
data_dir_path = '../../data/'
bias_corrected_data_dir_path = os.path.join(data_dir_path, 'preprocessed/0_bias_corrected/USZ')
raw_data_dir_path = os.path.join(data_dir_path, 'raw/USZ')

In [24]:
fps = glob.glob(os.path.join(bias_corrected_data_dir_path, '*'))

In [25]:
fps

['../../data/preprocessed/0_bias_corrected/USZ/10107525-MCA-new-Colors',
 '../../data/preprocessed/0_bias_corrected/USZ/10811755_CC_AComm_NEW',
 '../../data/preprocessed/0_bias_corrected/USZ/60213089-MCA-ICA-NEW',
 '../../data/preprocessed/0_bias_corrected/USZ/10764179-MCA-NEW_DONE',
 '../../data/preprocessed/0_bias_corrected/USZ/11096773_IB_PComm',
 '../../data/preprocessed/0_bias_corrected/USZ/60201588-MCA-NEW_DONE',
 '../../data/preprocessed/0_bias_corrected/USZ/10731286-ICA-links-NEW',
 '../../data/preprocessed/0_bias_corrected/USZ/11047954_LM_Pericallosa',
 '../../data/preprocessed/0_bias_corrected/USZ/10104830_ZG_ComplexICA_Checked',
 '../../data/preprocessed/0_bias_corrected/USZ/10299485_BMM_AComm_MCA_Checked',
 '../../data/preprocessed/0_bias_corrected/USZ/10266790_CL_ICA_Checked',
 '../../data/preprocessed/0_bias_corrected/USZ/10599468-AComm-new',
 '../../data/preprocessed/0_bias_corrected/USZ/10019209_EK_MCA_Checked',
 '../../data/preprocessed/0_bias_corrected/USZ/10986737_SH

In [28]:
missing_seg_mask = [ ]
for fp in fps:
    scans = glob.glob(os.path.join(fp, '*.nii.gz'))
    
    if len(scans) == 1:
        missing_seg_mask.append(fp)
        
    elif len(scans) == 0:
        print(f'{fp} has no scans at all')

In [29]:
missing_seg_mask

[]